In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import glob
import re 
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
import pandas as pd
import os
import torch
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision
from sklearn.metrics import classification_report
from sklearn import preprocessing
import datetime
import sklearn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import os
import shutil
import random
import cv2
import glob
import tqdm

In [ ]:
class TimeDistributed(nn.Module):
    def __init__(self, module, batch_first=False):
        super(TimeDistributed, self).__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, x):

        if len(x.size()) <= 2:
            return self.module(x)

        # Squash samples and timesteps into a single axis
        x_reshape = x.contiguous().view(-1, x.size(-1))  # (samples * timesteps, input_size)

        y = self.module(x_reshape)

        # We have to reshape Y
        if self.batch_first:
            y = y.contiguous().view(x.size(0), -1, y.size(-1))  # (samples, timesteps, output_size)
        else:
            y = y.view(-1, x.size(1), y.size(-1))  # (timesteps, samples, output_size)

        return y

In [ ]:
class Bottleneck(nn.Module):
    """
    Bottleneckを使用したresidual blockクラス
    """
    def __init__(self, indim, outdim, is_first_resblock=False):
        super(Bottleneck, self).__init__()
        self.is_dim_changed = (indim != outdim)
        # W, Hを小さくしてCを増やす際はstrideを2にする +
        # projection shortcutを使う様にセット
        if self.is_dim_changed:
            if is_first_resblock:
                # 最初のresblockは(W､ H)は変更しないのでstrideは1にする
                stride = 1
            else:
                stride = 2
            self.shortcut = nn.Conv2d(indim, outdim, 1, stride=stride)
        else:
            stride = 1
        
        dim_inter = int(outdim / 4)
        self.conv1 = nn.Conv2d(indim, dim_inter , 1)
        self.bn1 = nn.BatchNorm2d(dim_inter)
        self.conv2 = nn.Conv2d(dim_inter, dim_inter, 3,
                               stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(dim_inter)
        self.conv3 = nn.Conv2d(dim_inter, outdim, 1)
        self.bn3 = nn.BatchNorm2d(outdim)
        self.relu = nn.ReLU(inplace=True)
        

    def forward(self, x):
        shortcut = x
  
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)
        
        # Projection shortcutの場合
        if self.is_dim_changed:
            shortcut = self.shortcut(x)

        out += shortcut
        out = self.relu(out)

        return out


class ResNet50(nn.Module):
    
    def __init__(self): 
          
        super(ResNet50, self).__init__()
        
        # Due to memory limitation, images will be resized on-the-fly.
        self.upsampler = nn.Upsample(size=(224, 224))

        # Prior block
        self.layer_1 = nn.Conv2d(3, 64, 7, padding=3, stride=2)
        self.bn_1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(2, 2)
        
        # Residual blocks
        self.resblock1 = Bottleneck(64, 256, True)
        self.resblock2 = Bottleneck(256, 256)
        self.resblock3 = Bottleneck(256, 256)
        self.resblock4 = Bottleneck(256, 512)
        self.resblock5 = Bottleneck(512, 512)
        self.resblock6 = Bottleneck(512, 512)
        self.resblock7 = Bottleneck(512, 512)
        self.resblock8 = Bottleneck(512, 1024)
        self.resblock9 = Bottleneck(1024, 1024)
        self.resblock10 =Bottleneck(1024, 1024)
        self.resblock11 =Bottleneck(1024, 1024)
        self.resblock12 =Bottleneck(1024, 1024)
        self.resblock13 =Bottleneck(1024, 1024)
        self.resblock14 =Bottleneck(1024, 2048)
        self.resblock15 =Bottleneck(2048, 2048)
        self.resblock16 =Bottleneck(2048, 2048)
        
        # Postreior Block
        #self.glob_avg_pool = nn.AdaptiveAvgPool2d((1, 1))        
        #self.fc = nn.Linear(2048, 4)

    def forward(self, x):
        x = self.upsampler(x)
        
        # Prior block
        x = self.relu(self.bn_1(self.layer_1(x)))
        x = self.pool(x)
        
        # Residual blocks
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.resblock3(x)
        x = self.resblock4(x)
        x = self.resblock5(x)
        x = self.resblock6(x)
        x = self.resblock7(x)
        x = self.resblock8(x)
        x = self.resblock9(x)
        x = self.resblock10(x)
        x = self.resblock11(x)
        x = self.resblock12(x)
        x = self.resblock13(x)
        x = self.resblock14(x)
        x = self.resblock15(x)
        x = self.resblock16(x)
        
        # Postreior Block
        #x = self.glob_avg_pool(x)
        return x

In [ ]:
net = ResNet50()
device=torch.device('cuda')
net.cuda()


ResNet50(
  (upsampler): Upsample(size=(224, 224), mode=nearest)
  (layer_1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (resblock1): Bottleneck(
    (shortcut): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (resblock2): Bottleneck(
    (conv1): Conv

In [ ]:
original_dir = "drive/MyDrive/HMDB_CNNdata"
store_folder = "drive/MyDrive/late_fusion"

listofcats = os.listdir(original_dir)
data = []
if ".DS_Store" in listofcats:
  listofcats.remove(".DS_Store")
print(listofcats)
for cat in listofcats:
  print(cat)
  path2cat = os.path.join(original_dir + '/' + str(cat))
  listofts = os.listdir(path2cat)
  if ".DS_Store" in listofts:
    listofts.remove(".DS_Store")
  for i,ts in enumerate(listofts):
    path2ts = os.path.join(original_dir + '/' + str(cat) + '/' + str(ts))
    listofimgs = os.listdir(path2ts)
    if ".DS_Store" in listofimgs:
      listofimgs.remove(".DS_Store")
    if len(listofimgs) != 16:
      listofts.remove(ts)
    listofimgspath = sorted(glob.glob(path2ts + '/*.jpg'))
    #lsitifimgs = sorted(listofimgs)
    #print(listofimgspath)

    for path2img in listofimgspath:
      img = cv2.imread(path2img)
      img = TF.to_tensor(img)
      img = img[np.newaxis,:,:]
      img = img.to(device) 
      output = net(img) 
      #print(output.shape)
      #print(type(data))
      data.append(output)
    data = np.array(data)
    
    
    np.save(store_folder + '/' + str(cat) +  '/test' + str(i),data)
    data = []

['run', 'clap', 'brush_hair', 'smoke']
run
clap
brush_hair
smoke


In [ ]:
import os
import shutil
import random


def image_dir_train_test_sprit(original_dir, base_dir, train_size=0.8):
    '''
    画像データをトレインデータとテストデータにシャッフルして分割します。フォルダもなければ作成します。

    parameter
    ------------
    original_dir: str
      オリジナルデータフォルダのパス その下に各クラスのフォルダがある
    base_dir: str
      分けたデータを格納するフォルダのパス　そこにフォルダが作られます
    train_size: float
      トレインデータの割合
    '''
    try:
        os.mkdir(base_dir)
    except FileExistsError:
        print(base_dir + "は作成済み")

    #クラス分のフォルダ名の取得
    dir_lists = os.listdir(original_dir)
    dir_lists = [f for f in dir_lists if os.path.isdir(os.path.join(original_dir, f))]
    original_dir_path = [os.path.join(original_dir, p) for p in dir_lists]

    num_class = len(dir_lists)

    # フォルダの作成(トレインとバリデーション)
    try:
        train_dir = os.path.join(base_dir, 'train')
        os.mkdir(train_dir)
    except FileExistsError:
        print(train_dir + "は作成済み")

    try:
        validation_dir = os.path.join(base_dir, 'validation')
        os.mkdir(validation_dir)
    except FileExistsError:
        print(validation_dir + "は作成済み")

    #クラスフォルダの作成
    train_dir_path_lists = []
    val_dir_path_lists = []
    for D in dir_lists:
        train_class_dir_path = os.path.join(train_dir, D)
        try:
            os.mkdir(train_class_dir_path)
        except FileExistsError:
            print(train_class_dir_path + "は作成済み")
        train_dir_path_lists += [train_class_dir_path]
        val_class_dir_path = os.path.join(validation_dir, D)
        try:
            os.mkdir(val_class_dir_path)
        except FileExistsError:
            print(val_class_dir_path + "は作成済み")
        val_dir_path_lists += [val_class_dir_path]


    #元データをシャッフルしたものを上で作ったフォルダにコピーします。
    #ファイル名を取得してシャッフル
    for i,path in enumerate(original_dir_path):
        files_class = os.listdir(path)
        random.shuffle(files_class)
        # 分割地点のインデックスを取得
        num_bunkatu = int(len(files_class) * train_size)
        #トレインへファイルをコピー
        for fname in files_class[:num_bunkatu]:
            src = os.path.join(path, fname)
            dst = os.path.join(train_dir_path_lists[i], fname)
            shutil.copy(src, dst)
        #valへファイルをコピー
        for fname in files_class[num_bunkatu:]:
            src = os.path.join(path, fname)
            dst = os.path.join(val_dir_path_lists[i], fname)
            shutil.copyfile(src, dst)
        print(path + "コピー完了")

    print("分割終了")


def main():
    original_dir = "drive/MyDrive/HMDB_CNNdata"
    base_dir = "sprit_data_late"
    train_size = 0.8
    image_dir_train_test_sprit(original_dir, base_dir, train_size)



main()

sprit_data_lateは作成済み
sprit_data_late/trainは作成済み
sprit_data_late/validationは作成済み
sprit_data_late/train/runは作成済み
sprit_data_late/validation/runは作成済み
sprit_data_late/train/clapは作成済み
sprit_data_late/validation/clapは作成済み
sprit_data_late/train/brush_hairは作成済み
sprit_data_late/validation/brush_hairは作成済み
sprit_data_late/train/smokeは作成済み
sprit_data_late/validation/smokeは作成済み


IsADirectoryError: ignored

In [ ]:
transform_train = transforms.Compose([
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


train_data_dir = 'drive/MyDrive/sprit_data/train'
valid_data_dir = 'drive/MyDrive/sprit_data/validation'

# training set
trainset = torchvision.datasets.ImageFolder(train_data_dir, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True)

# validation set
validset = torchvision.datasets.ImageFolder(valid_data_dir, transform=transform_valid)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=1, shuffle=False)

In [ ]:
import cv2
import torchvision.transforms.functional as TF
img = cv2.imread('/content/drive/MyDrive/HMDB_jpg/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_0/frame0.jpg')
img = cv2.resize(img,(224,224))

print(img.shape)
img = TF.to_tensor(img)
img = img[np.newaxis,:,:]

img = img.to(device)
print(img.shape)
output = net(img)
print(output)
print(output.shape)

(224, 224, 3)
torch.Size([1, 3, 224, 224])
tensor([[[[0.5677, 0.7359, 1.6069,  ..., 0.0000, 0.6679, 0.4289],
          [0.7836, 0.0000, 0.0000,  ..., 0.0000, 1.3178, 0.9275],
          [0.3393, 1.7268, 0.0000,  ..., 0.0000, 0.0000, 0.7083],
          ...,
          [0.6627, 4.2658, 0.9431,  ..., 0.5008, 3.3301, 0.8779],
          [2.9234, 0.1424, 1.9738,  ..., 0.1039, 0.2358, 2.0286],
          [0.8158, 1.9531, 0.6224,  ..., 0.0000, 2.3881, 0.0000]],

         [[1.8752, 0.0000, 0.0000,  ..., 3.1828, 0.5529, 0.0000],
          [0.5036, 0.0000, 1.6022,  ..., 2.8216, 0.0000, 0.0000],
          [0.4407, 2.2957, 0.1571,  ..., 1.9813, 1.3899, 0.0000],
          ...,
          [0.1119, 1.4506, 0.8511,  ..., 1.2511, 0.4738, 0.9795],
          [0.4719, 2.8613, 1.0395,  ..., 0.3868, 0.0000, 0.4820],
          [0.0000, 0.3942, 0.0000,  ..., 0.5566, 0.0000, 0.0000]],

         [[2.2534, 0.3338, 0.9953,  ..., 1.3595, 1.4478, 0.4259],
          [0.4998, 0.2001, 0.0000,  ..., 2.6844, 2.8609, 1.6079],

In [ ]:
for data,labels in train_loader:
  data,labels = data.to(device), labels.to(device)
  print(data)
  output = net(data)
  print(output.shape)

ストリーミング出力は最後の 5000 行に切り捨てられました。
tensor([[[[ 0.4508,  0.4337,  0.3994,  ...,  1.9235,  1.8722,  1.9749],
          [ 0.4508,  0.4337,  0.3994,  ...,  1.9235,  1.8379,  1.9578],
          [ 0.4337,  0.4337,  0.4337,  ...,  1.9064,  1.8379,  1.9235],
          ...,
          [-1.3644, -1.3473, -1.3473,  ..., -0.9534, -0.9534, -0.9363],
          [-1.3644, -1.3644, -1.3644,  ..., -1.0048, -1.0048, -0.9877],
          [-1.3815, -1.3815, -1.3644,  ..., -0.9705, -0.9877, -0.9534]],

         [[ 0.3452,  0.3277,  0.3452,  ...,  2.0959,  1.9909,  2.0784],
          [ 0.3452,  0.3277,  0.3452,  ...,  2.0434,  1.9384,  2.0084],
          [ 0.3277,  0.3277,  0.3277,  ...,  2.0084,  1.8859,  1.9384],
          ...,
          [-0.8627, -0.8452, -0.8452,  ..., -1.1078, -1.1078, -1.0903],
          [-0.8627, -0.8627, -0.8627,  ..., -1.1078, -1.1078, -1.0903],
          [-0.8803, -0.8803, -0.8627,  ..., -1.0728, -1.0903, -1.0553]],

         [[ 0.0605,  0.0431,  0.0431,  ...,  2.1346,  2.0474,  2.1346]